# REAL ESTATE DATA API

## Imports

In [7]:
import subprocess
import warnings
from datetime import datetime

import pandas as pd
import requests

warnings.filterwarnings('ignore')


## Generating Dataset

### API Props

In [36]:
client_secret = "secret_9d2aeb1d94d5f400c8128c5199a8d8de"
client_id = "client_2bf50cfd2349026b489480f4c464f395"

raw = subprocess.check_output("node getToken.js", shell=True)
bearer = raw.decode("utf-8").strip()

In [63]:
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer}"
}

In [62]:
# URLs
listings = "https://api.domain.com.au/v1/listings/residential/_search"

In [64]:
req_body = lambda pg_no: {
  "listingType":"Sale",
  "propertyTypes":[
    "House"
  ],
  "minBedrooms":2,
  "maxBedrooms":5,
  "minPrice": 350000,
  "maxPrice": 800000,
  "listingAttributes": [
    "HasPhotos"
  ], 
  "locations":[
    {
      "state":"VIC",
      "region":"Melbourne Region",
      "area":"",
      "suburb":"",
      "postCode":""
    }
  ],
  "excludePriceWithheld": True,
  "excludeDepositTaken": True,
  "pageSize": 200,
  "pageNumber": pg_no
}

### Health Check

In [65]:
resp = requests.post(url=listings, json=req_body(1), headers=headers)
resp_json = resp.json()
print(resp)
print(resp_json)

<Response [200]>
[{'type': 'Project', 'listings': [{'listingType': 'NewHomes', 'id': 2017938391, 'projectId': 5614, 'advertiser': {'type': 'Agency', 'id': 25325, 'name': 'Lendlease Retirement Living', 'logoUrl': 'https://images.domain.com.au/img/Agencys/25325/logo_25325.jpeg', 'preferredColourHex': '#DDDDDD', 'bannerUrl': 'https://images.domain.com.au/img/Agencys/25325/banner_25325.jpeg', 'contacts': [{'name': 'Jane Flynn', 'photoUrl': 'https://images.domain.com.au/img/25325/contact_1864221.png?mod=220715-163915'}]}, 'priceDetails': {'price': 485000, 'priceFrom': 485000, 'priceTo': 485000, 'displayPrice': '$485,000'}, 'media': [{'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2017938391_1_1_220715_060553-w3224-h2160'}, {'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2017938391_2_1_220715_060553-w3224-h2160'}, {'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2017938391_3_1_220715_060553-w3224-h216

### Dataframe Writer

In [11]:
def update_df(df, resp_json):
  for listing in resp_json:
    if "listing" in listing:
      house = listing['listing']
      full_dict = {}
      full_dict.update({k: v for k,v in house.items() if k in ['listingType', 'id', 'dateListed']})
      full_dict.update(house['priceDetails'])
      full_dict.update({k: v for k,v in house['propertyDetails'].items() if k in ['propertyType', 'bathrooms', 'bedrooms', 'carspaces', 'unitNumber', 'streetNumber', 'street', 'area', 'region', 'suburb', 'postcode', 'displayableAddress', 'landArea']})
      df = df.append(full_dict, ignore_index=True)
  return df

### Main

In [12]:
if __name__ == "__main__":
    FULL_DF = pd.DataFrame()

    for i in range(1, 10):
        resp = requests.post(url=listings, json=req_body(i), headers=headers)
        resp_json = resp.json()
        
        # process data and update df
        FULL_DF = update_df(FULL_DF, resp_json)

        if len(resp_json) < 200:
            break

In [13]:
FULL_DF

""


### Save to CSV

In [198]:
now = datetime.now().strftime("%Y-%m-%d")
FULL_DF.to_csv(f"houses-{now}.csv")


,listingType,id,dateListed,price,priceFrom,priceTo,displayPrice,propertyType,bathrooms,bedrooms,carspaces,unitNumber,streetNumber,street,area,region,suburb,postcode,displayableAddress,landArea
0,Sale,2017936760,2022-07-14T21:43:17,620000,620000,650000,"$620,000 to $650,000",House,2.0,4.0,2.0,,48,Mirka Way,Wyndham City Council - Greater Area,Melbourne Region,POINT COOK,3030,"48 Mirka Way, Point Cook",324.0
1,Sale,2017874360,2022-06-16T11:02:22,755000,755000,755000,"$755,000",House,2.0,4.0,1.0,,15,Oakwood Drive,Frankston Council - Greater Area,Melbourne Region,CARRUM DOWNS,3201,"15 Oakwood Drive, Carrum Downs",703.0
2,Sale,2017872934,2022-06-15T16:25:29,700000,700000,700000,"$700,000",House,2.0,4.0,2.0,,2,Koombahla Court,Wyndham City Council - Greater Area,Melbourne Region,WERRIBEE,3030,"2 Koombahla Court, Werribee",393.0
3,Sale,2017857720,2022-06-08T13:24:41,700000,700000,700000,"$700,000",House,2.0,4.0,3.0,,26,Brentwood Crescent,Frankston Council - Greater Area,Melbourne Region,FRANKSTON,3199,"26 Brentwood Crescent, Frankston",559.0
4,Sale,2017819204,2022-05-20T16:18:59,670000,670000,670000,"$670,000.00",House,2.0,4.0,4.0,,23,Elstar Road,Casey City - Greater Area,Melbourne Region,NARRE WARREN,3805,"23 Elstar Road, Narre Warren",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Sale,2015686942,2019-09-26T16:50:15,553000,553000,553000,"$553,000",House,2.0,3.0,2.0,,,,Melton City - Greater Area,Melbourne Region,PLUMPTON,3335,Plumpton,381.0
271,Sale,2015686889,2019-09-26T16:42:23,553000,553000,553000,"$553,000",House,2.0,4.0,2.0,,,,Melton City - Greater Area,Melbourne Region,PLUMPTON,3335,Plumpton,382.0
272,Sale,2015686832,2019-09-26T16:36:02,531000,531000,531000,"$531,000",House,2.0,3.0,1.0,,,,Melton City - Greater Area,Melbourne Region,PLUMPTON,3335,Plumpton,320.0
273,Sale,2015647965,2019-09-16T12:12:03,690000,690000,740000,"$690,000 to $740,000",House,1.0,3.0,2.0,,13,Jeffery Ave,Dandenong - Greater Area,Melbourne Region,NOBLE PARK,3174,"13 Jeffery Ave, Noble Park",529.0


## Other Fields # TODO

In [ ]:
# Distance to cities
# Distance to schools
# Median House Price
# Listing URL, https://developer.domain.com.au/docs/latest/apis/pkg_agents_listings/references/listings_get, "seoUrl"

## Data Visualisation #TODO

In [195]:
# Choose CSV file (default latest)
# Plotly graphs, scatterplots - ensure popup thingos include address
# Geo plot?

'2022-07-19'